<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/%EC%8B%9C%EA%B0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import argparse
import os
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
import torch.nn as nn
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# PyTorch에서 MNIST 데이터 로드
transform = transforms.Compose([transforms.ToTensor()])
mnist_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
data_loader = torch.utils.data.DataLoader(mnist_data, batch_size=1000, shuffle=True)

In [ ]:
imgs = []

for img, label in data_loader:
    # label이 0인 이미지만 처리합니다.
    # img = img[label == 0]
    img = img.flatten(1)
    imgs.append(img.numpy())

imgs = np.concatenate(imgs, axis=0)

print(imgs.shape)

In [ ]:
# 축소 및 밀도 추정
pca2 = PCA(n_components=2)
data_pca2 = pca2.fit_transform(np.stack(imgs))
data_pca2.shape

kde = gaussian_kde(data_pca2.T)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 2차원 그리드 생성
x = np.linspace(data_pca2[:, 0].min(), data_pca2[:, 0].max(), 100)
y = np.linspace(data_pca2[:, 1].min(), data_pca2[:, 1].max(), 100)
X, Y = np.meshgrid(x, y)

# 그리드 상의 각 지점에서 밀도 계산
positions = np.vstack([X.ravel(), Y.ravel()])
density = kde(positions).reshape(X.shape)

# 3차원 그래프로 시각화
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X, Y, density, cmap='viridis', edgecolor='none')
ax.set_xlabel('PCA 1')
ax.set_ylabel('PCA 2')
ax.set_zlabel('Density')
plt.title('Gaussian KDE of MNIST Digits (Label 0) in 3D')
plt.colorbar(surf)
plt.show()

In [ ]:
# pca3 = PCA(n_components=3)
# data_pca3 = pca3.fit_transform(np.stack(imgs))
# data_pca3.shape

# kde = gaussian_kde(data_pca3.T)

# # 3D 그리드 생성
# x = np.linspace(data_pca3[:, 0].min(), data_pca3[:, 0].max(), 100)
# y = np.linspace(data_pca3[:, 1].min(), data_pca3[:, 1].max(), 100)
# z = np.linspace(data_pca3[:, 2].min(), data_pca3[:, 2].max(), 100)
# X, Y, Z = np.meshgrid(x, y, z)
# positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel()])

# # 밀도 계산
# density = kde(positions).reshape(X.shape)

# # 밀도가 높은 지점만 시각화하기 위한 임계값 설정
# threshold = density.mean()

# # 3D 그래프 시각화
# fig = plt.figure(figsize=(10, 7))
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(X[density > threshold], Y[density > threshold], Z[density > threshold], c=density[density > threshold], cmap='viridis')
# plt.title('Gaussian KDE of MNIST Digits (Label 0)')
# plt.show()

# 생성모델


In [ ]:
n_epochs = 200  # 학습 에폭 수
batch_size = 64  # 배치 크기
lr = 0.0002  # 학습률
b1 = 0.5  # adam 최적화기의 베타1 값
b2 = 0.999  # adam 최적화기의 베타2 값
n_cpu = 8  # 배치 생성 시 사용할 CPU 쓰레드 수
latent_dim = 100  # 잠재 공간의 차원 수
img_size = 28  # 이미지의 한 변의 크기
channels = 1  # 이미지의 채널 수
sample_interval = 400  # 이미지 샘플링 간격

img_shape = (channels, img_size, img_size)

# Generator 클래스 정의
class Generator(nn.Module):
    def __init__(self, latent_dim, img_shape):
        super(Generator, self).__init__()

        def block(in_dim, out_dim, normalize=True):
            layers = [nn.Linear(in_dim, out_dim)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_dim, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *img_shape)
        return img

# 모델 인스턴스 생성
generator = Generator(latent_dim=latent_dim, img_shape=img_shape)

In [ ]:
# 모델 로드 (이전에 저장된 모델의 경로를 지정하세요)
model_path = '/content/drive/MyDrive/Colab Notebooks/model/gan198.pt'
generator = torch.load(model_path, map_location=torch.device('cpu'))  # CPU로 매핑
generator.eval()

In [ ]:
# z = torch.randn(1, latent_dim, device=DEVICE)
z = torch.randn(1, latent_dim)
z = generator(z)


# img_tensor = torch.rand(1, 1, 28, 28)  # 실제 코드에서는 GAN으로 생성된 이미지 텐서 사용

# 생성된 이미지 텐서를 가져옵니다. 예: img_tensor = 생성한 이미지 텐서
# 예를 들어, img_tensor 변수가 이미 생성된 이미지를 가지고 있다고 가정합니다.

def imshow(img_tensor):
    img = img_tensor.squeeze()  # 차원 축소: [1, 1, 28, 28] -> [28, 28]
    img = img.detach()
    plt.imshow(img, cmap='gray')  # 흑백 이미지로 출력
    plt.axis('off')  # 축 제거
    plt.show()

# 이미지 출력
imshow(z)

In [ ]:
z = torch.randn(100000, latent_dim)
z = generator(z)

In [ ]:
z.shape

In [ ]:
z = z.flatten(1)
z.shape

In [ ]:
z = z.detach()
z = z.numpy()

In [ ]:
# 축소 및 밀도 추정
pca2 = PCA(n_components=2)
data_pca2t = pca2.fit_transform(np.stack(z))
data_pca2t.shape

kde = gaussian_kde(data_pca2t.T)

In [ ]:
# 2차원 그리드 생성
x = np.linspace(data_pca2t[:, 0].min(), data_pca2t[:, 0].max(), 100)
y = np.linspace(data_pca2t[:, 1].min(), data_pca2t[:, 1].max(), 100)
X, Y = np.meshgrid(x, y)

# 그리드 상의 각 지점에서 밀도 계산
positions = np.vstack([X.ravel(), Y.ravel()])
density = kde(positions).reshape(X.shape)

# 3차원 그래프로 시각화
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X, Y, density, cmap='viridis', edgecolor='none')
ax.set_xlabel('PCA 1')
ax.set_ylabel('PCA 2')
ax.set_zlabel('Density')
plt.title('Gaussian KDE of MNIST Digits (Label 0) in 3D')
plt.colorbar(surf)
plt.show()